# Yoseph Yan, ypy4, March 5, 2024, Intelligent Autonomous Systems

In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.cluster import KMeans
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Function that tests the model
def test(clusters, pi, A, B):
    # Initializes the parameter
    param = 0

    # Number of observations
    T = len(clusters)

    # Initializes alpha
    alpha = np.zeros((N, T))

    # Computes alpha
    alpha[:, 0] = (pi * B[:, clusters[0]])

    # Loops through the observations
    for t in range(1, T):
        # Computes alpha
        alpha[:, t] = ((alpha[:, t - 1]).dot(A)) * B[:, clusters[t]]
        asum = alpha[:, t].sum()
        alpha[:, t] /= asum
        param += np.log(asum)

    # Returns the result
    return param

In [3]:
# Defines the path to the test data
testpath = "ECE5242Proj2-test-selected" 

# Loads the model
with open('model/model.pickle', 'rb') as file:
    dictionary = pickle.load(file)
    
# Loads kmeans
with open('model/kmeans.pickle', 'rb') as file:
    kmeans = pickle.load(file)

# Initializes the number of hidden states and observation clusters
N = 15
M = 75
    
# Loops through the test files
for testfile in os.listdir(testpath):
    # Full path to the files in the test folder
    testfilepath = os.path.join(testpath, testfile)

    # Reads the data from the files
    data = pd.read_csv(testfilepath, sep='\t', header=None, names=['ts', 'Wx', 'Wy', 'Wz', 'Ax', 'Ay', 'Az'])
    data = data.drop(columns=['ts'], axis=1)

    # Computes the kmeans clustering
    clusters = kmeans.predict(data)

    # Initializes the results
    results = {}

    # Loops through the labels and computes the test
    for current_gesture in dictionary.keys():
        # Initializes pi, a, b
        pi = dictionary[current_gesture]["pi"]
        a = dictionary[current_gesture]["A"]
        b = dictionary[current_gesture]["B"]
        
        # Computes test
        test_results = test(clusters, pi, a, b)
        results[current_gesture] = test_results

    # Sorts to get 3 highest gestures
    gesture = sorted(results.items(), key=lambda x: -x[1])
    
    # Prints the test results
    print("Prediction for " + testfile + ": (1) " + gesture[0][0] + ", (2) " + gesture[1][0] + ", (3) " + gesture[2][0])

Prediction for test7.txt: (1) wave, (2) eight, (3) beat4
Prediction for test6.txt: (1) eight, (2) inf, (3) beat4
Prediction for test4.txt: (1) beat3, (2) beat4, (3) wave
Prediction for test5.txt: (1) circle, (2) beat4, (3) beat3
Prediction for test1.txt: (1) inf, (2) eight, (3) beat3
Prediction for test2.txt: (1) beat4, (2) beat3, (3) wave
Prediction for test3.txt: (1) inf, (2) eight, (3) beat3
Prediction for test8.txt: (1) beat3, (2) beat4, (3) wave
